# 1. Data Merging and Integration


This notebook will provide an overview of the dataset and demonstrate how to link the data with external resources such as statistics from MLB.com and FanGraphs. The notebook will use several external scripts that will not be explained in-depth; this notebook is simply to provide an overview of the process.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from data_integration import *

We begin by loading the `twtc` dataset as well as the dataset containing statistics for many players within the dataset. 

We can see that the data has information like player names and IDs, birthdates, when (or if) the player player made his MLB debut, their age at the time of the report, and so on. There are 9200 rows, of which nearly 8000 have player statistics.

In [2]:
twtc = load_twtc()

stats = load_stats_df(twtc)
stats = stats.drop(['fg_season_id', 'am_season_id'], axis=1)

df = pd.concat([twtc, stats], axis=1)

df = fill_missing_data(df, stats.columns)
df = df.replace('*.**', np.nan).dropna(subset=stats.columns) # some bad values from MLBAM

print(df.shape)
print(f'Non-empty rows: {(df[stats.columns].sum(axis=1) != 0).sum()}')
df.head()

(9214, 65)
Non-empty rows: 7969


,name,key_mlbam,key_fangraphs,key_bbref,key_bbref_minors,key_uuid,mlb_played_first,birthdate,debut_age,age,...,H,BB_pit,Age,3B,SO_pit,OBP,AVG,RBI,R,WHIP
0,**Luke Heimlich,-1,18360,NaN,NaN,NaN,NaN,NaN,NaN,22.3,...,82.0,0.0,—,0.0,0.0,0.345679,0.281787,47.0,41.0,0.000000
1,**Noah Song,679977,sa917943,NaN,NaN,NaN,NaN,NaN,NaN,22.0,...,75.0,0.0,—,3.0,0.0,0.349081,0.235849,33.0,47.0,0.880000
2,A.J. Alexy,669935,NaN,NaN,alexy-000aj-,c5bd95ff-593c-4875-b48e-012da0caca64,NaN,1998-04-21T00:00:00.000Z,NaN,19.7,...,0.0,3.0,0,0.0,12.0,0.000000,0.000000,0.0,0.0,1.460000
3,A.J. Alexy,669935,NaN,NaN,alexy-000aj-,c5bd95ff-593c-4875-b48e-012da0caca64,NaN,1998-04-21T00:00:00.000Z,NaN,18.7,...,0.0,3.0,0,0.0,12.0,0.000000,0.000000,0.0,0.0,1.460000
4,A.J. Cole,595918,11467,coleaj01,cole--001aj-,0596c185-ae62-436a-92b9-79b9a145d64d,2015.0,1992-01-05T00:00:00.000Z,23.0,22.0,...,0.0,15.0,—,0.0,61.0,0.000000,0.000000,0.0,0.0,1.323944


Here is a full look at all 65 columns:

In [3]:
df.columns

Index(['name', 'key_mlbam', 'key_fangraphs', 'key_bbref', 'key_bbref_minors',
       'key_uuid', 'mlb_played_first', 'birthdate', 'debut_age', 'age', 'year',
       'primary_position', 'eta', 'report', 'Arm', 'Changeup', 'Control',
       'Curveball', 'Cutter', 'Fastball', 'Field', 'Hit', 'Power', 'Run',
       'Slider', 'Splitter', 'source', 'name_count', 'clean_name',
       'mlbam_candidate', 'old_mlbam', 'fg_season_id', 'am_season_id', 'AB',
       'HR_pit', 'W', 'Pitches', 'HR', 'SV', 'SB', 'SLG', 'L', 'GP', 'ERA',
       'IP', 'BS', 'Hld', 'CS', '2B', 'Level', 'Age_pit', 'OPS', 'TBF', 'BB',
       'SO', 'H', 'BB_pit', 'Age', '3B', 'SO_pit', 'OBP', 'AVG', 'RBI', 'R',
       'WHIP'],
      dtype='object')

Let's take a look at a sample report for Vladimir Guerrero Jr.

In [4]:
player = df.query('name == "Vladimir Guerrero Jr"').head(1)

print(player[['name', 'year']])

                      name  year
8802  Vladimir Guerrero Jr  2018


We can see his report for that year, as well as the publishing source. Note that a player can have multiple reports in the same year from difference sources.

In [5]:
print(player.report.values[0])
print(f'\nSource: {player.source.values[0]}')

Guerrero was identified as an elite talent for his age years before the Jays signed him at age 16, evident from a similar advanced feel for hitting and raw power of his father. Unlike his Hall of Fame father, Vlad Jr. has generally developed earlier--physically looking too big for third base as a teenager and polishing his tools at a very young stage. Whether Vlad Jr. settles as a fringy third baseman or a first basemen/designated hitter is up for debate, but his easy plus hit and power tools (with ceiling for more) are not and will make his ascent to the big leagues a quick one.

Source: fg_2018 Report


We can also take a look at his numeric grades.

For pitchers, hitting-related grades will be zeroed; for hitters, the same is true for pitching-related grades.

In [6]:
grade_cols = ['Changeup', 'Control','Curveball', 'Cutter', 'Fastball', 'Slider', 'Splitter',
              'Arm', 'Field', 'Hit', 'Power', 'Run']
player[grade_cols]

,Changeup,Control,Curveball,Cutter,Fastball,Slider,Splitter,Arm,Field,Hit,Power,Run
8802,0,0,0,0,0,0,0.0,60,50,65,70,30


We'll save this to a csv and move on to the next notebook.

In [7]:
df[(df[stats.columns].sum(axis=1) != 0) & (df.Level != 'MLB')].to_csv('joined_stats.csv', index=False)